<a href="https://colab.research.google.com/github/rugkfl/study_AIs/blob/main/docs/quests/MLs/LetalCarOfContractType.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 데이터

In [92]:
import pandas as pd
df_LCCT = pd.read_csv('LetalCarOfContractType.csv')
df_LCCT[:5]

,id,type_of_contract,type_of_contract2,channel,datetime,Term,payment_type,product,amount,state,overdue_count,overdue,credit rating,bank,cancellation,age,Mileage
0,66758234,렌탈,Normal,서비스 방문,2019-10-20,60,CMS,K1,96900,계약확정,0,없음,9.0,새마을금고,정상,43.0,1862.0
1,66755948,렌탈,Extension_Rental,서비스 방문,2019-10-20,60,카드이체,K1,102900,계약확정,0,없음,2.0,현대카드,정상,62.0,2532.0
2,66756657,렌탈,Normal,홈쇼핑/방송,2019-10-20,60,CMS,K1,96900,계약확정,0,없음,8.0,우리은행,정상,60.0,2363.0
3,66423450,멤버십,TAS,렌탈재계약,2019-10-20,12,CMS,K1,66900,계약확정,0,없음,5.0,농협은행,정상,60.0,2449.0
4,66423204,멤버십,TAS,렌탈재계약,2019-10-20,12,CMS,K1,66900,해약확정,12,있음,8.0,농협은행,해약,51.0,1942.0


In [50]:
df_LCCT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51304 entries, 0 to 51303
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 51304 non-null  int64  
 1   type_of_contract   51300 non-null  object 
 2   type_of_contract2  51303 non-null  object 
 3   channel            51304 non-null  object 
 4   datetime           51304 non-null  object 
 5   Term               51304 non-null  int64  
 6   payment_type       51304 non-null  object 
 7   product            51303 non-null  object 
 8   amount             51304 non-null  int64  
 9   state              51304 non-null  object 
 10  overdue_count      51304 non-null  int64  
 11  overdue            51302 non-null  object 
 12  credit rating      42521 non-null  float64
 13  bank               48544 non-null  object 
 14  cancellation       51279 non-null  object 
 15  age                40509 non-null  float64
 16  Mileage            405

## 데이터 전처리

In [52]:
df_LCCT_extract = df_LCCT.loc[:, ['age', 'Term', 'amount', 'overdue_count']]
df_LCCT_extract[:2]

,age,Term,amount,overdue_count
0,43.0,60,96900,0
1,62.0,60,102900,0


In [53]:
df_age = df_LCCT_extract['age'].dropna()
df_age.isnull().sum()

0

In [54]:
df_LCCT_extract['df_age'] = df_LCCT_extract['age'].dropna()
df_LCCT_extract[:2]

,age,Term,amount,overdue_count,df_age
0,43.0,60,96900,0,43.0
1,62.0,60,102900,0,62.0


In [64]:
df_LCCT_extract.dropna(inplace=True)

### 데이터 분리

In [65]:
features = df_LCCT_extract[['Term', 'amount', 'overdue_count']]
target = df_LCCT_extract['age']

In [109]:
from sklearn.model_selection import train_test_split
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.3, random_state=2)
features_train.shape, features_test.shape, target_train.shape, target_test.shape

((28356, 3), (12153, 3), (28356,), (12153,))

## 모델

In [110]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

In [111]:
model.fit(features, target)

LinearRegression()

## 평가

In [70]:
from sklearn.metrics import r2_score
target_train_predict = model.predict(features_train)

In [71]:
r2_score(target_train, target_train_predict)

-4.700106991850639e-05

In [77]:
round(-4.700106991850639e-05,2)

-0.0

In [72]:
target_test_predict = model.predict(features_test)
target_test.shape, target_test_predict.shape

((12153,), (12153,))

In [73]:
r2_score(target_test, target_test_predict)

0.00034347976917648015

## 전처리 적용

In [96]:
list_index = df_LCCT.isnull().query('age==True').index
features_null = df_LCCT.loc[list_index][['Term', 'amount', 'overdue_count']]

In [104]:
age_non = model.predict(features_null)

In [105]:
df_LCCT['age'].loc[list_index]=age_non

<ipython-input-105-eaebe357a0ce>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_LCCT['age'].loc[list_index]=age_non


In [108]:
df_LCCT['age'].loc[list_index]

7        46.583272
19       46.573996
21       46.581064
26       46.578413
27       47.074764
           ...    
51289    46.833928
51291    46.580180
51293    46.578413
51298    46.833928
51299    46.578413
Name: age, Length: 10795, dtype: float64

In [113]:
# 재사용 위해 model을 파일로 저장
import pickle # 메모리 인스턴스 클래스를 이진파일로 저장

In [114]:
with open('LetalCarOfContractType.pkl', 'wb') as pickle_file :
  pickle.dump(obj=model, file=pickle_file)

In [115]:
# csv 파일로 저장
df_LCCT.to_csv('LetalCarOfContractType_age_nonnull.csv', index=False, encoding='utf-8-sig')